> Summarized from this awesome video about the discussion of chains and agents: https://www.youtube.com/watch?v=bYLHklxEd_k&t=1801s


- Chains and agents represent two concepts in technology with distinct functionalities. Chains refer to linear sequences of events or processes that rely on the step-by-step execution of tasks, often requiring predefined inputs and producing predictable outcomes. 

- Agents, conversely, are autonomous systems capable of making decisions and performing tasks independently, adapting to new situations without explicit instructions. While chains excel in structured environments with clear rules, agents are designed to navigate complex, dynamic scenarios, learning and adjusting their behavior over time. The core difference lies in the level of autonomy and complexity each can handle, with agents typically seen as more advanced due to their self-governing nature.

In [26]:
# !pip install langchain
# !pip install langchain-openai
# !pip install langchainhub
# !pip install pypdf
# !pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 25.1 MB/s eta 0:00:000:00:01m eta 0:00:01


Making a reporting agent.

Let's build an LLM agent that can build simple reports for tasks performed just by reading through the code and files inside a folder + notes from Notion page or .txt file.

First, let's organize our thoughts, an agent will be composed of:

- Tool
- Base LLM
- Conditions for stoping and returning an output 

In [1]:
# import os

# # Set OPENAI API Key

# os.environ["OPENAI_API_KEY"] = "your openai key"

# OR (load from .env file)

from dotenv import load_dotenv
load_dotenv("./.env")

True

![](2023-10-27-19-28-10.png)

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
search = TavilySearchResults()

In [5]:
search.invoke("what is the weather in Portugal?")

[{'url': 'https://www.timeanddate.com/weather/portugal/lisbon/ext',
  'content': 'Currently: 59 °F. Passing clouds. (Weather station: Lisbon Portela Airport, Portugal). See more current weather Lisbon Extended Forecast with high and low temperatures °F Feb 12 - Feb 18 0.13 Lo:53 Thu, 15 Hi:64 15 Lo:51 Fri, 16 Hi:65 8 Lo:52 Sat, 17 Hi:67 5 Lo:52 Sun, 18 Hi:70 6 Feb 19 - Feb 25 Lo:54 Mon, 19 Hi:70 7 Lo:50 Tue, 20 Hi:68'},
 {'url': 'https://www.climatestotravel.com/climate/portugal',
  'content': 'When to go What to pack The seasons Winter, from December to February, is mild on the coast, even in the northern part, since the average temperature in January is around 10 °C (50 °F) in Porto, 12 °C (53.5 °F) in Lisbon, and 12.5 °C (54.5 °F) in Faro. In winter, there are periods of good weather when the Azores Anticyclone settles in the country.'},
 {'url': 'https://www.timeanddate.com/weather/portugal',
  'content': 'Weather in Portugal Time/General Weather Time Zone DST Changes Sun & Moon Ec

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://python.langchain.com/docs/get_started/introduction")
docs = loader.load()
docs

[Document(page_content="\n\n\n\n\nIntroduction | 🦜️🔗 Langchain\n\n\n\n\n\n\n\nSkip to main content🦜️🔗 LangChainDocsUse casesIntegrationsGuidesAPIMorePeopleVersioningChangelogContributingTemplatesCookbooksTutorialsYouTube🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS DocsChatSearchGet startedIntroductionInstallationQuickstartSecurityLangChain Expression LanguageGet startedWhy use LCELInterfaceStreamingHow toCookbookLangChain Expression Language (LCEL)ModulesModel I/ORetrievalAgentsChainsMoreLangServeLangSmithLangGraphGet startedIntroductionOn this pageIntroductionLangChain is a framework for developing applications powered by language models. It enables applications that:Are context-aware: connect a language model to sources of context (prompt instructions, few shot examples, content to ground its response in, etc.)Reason: rely on a language model to reason (about how to answer based on provided context, what actions to take, etc.)This framewor

In [13]:
docs[0]

Document(page_content="\n\n\n\n\nIntroduction | 🦜️🔗 Langchain\n\n\n\n\n\n\n\nSkip to main content🦜️🔗 LangChainDocsUse casesIntegrationsGuidesAPIMorePeopleVersioningChangelogContributingTemplatesCookbooksTutorialsYouTube🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS DocsChatSearchGet startedIntroductionInstallationQuickstartSecurityLangChain Expression LanguageGet startedWhy use LCELInterfaceStreamingHow toCookbookLangChain Expression Language (LCEL)ModulesModel I/ORetrievalAgentsChainsMoreLangServeLangSmithLangGraphGet startedIntroductionOn this pageIntroductionLangChain is a framework for developing applications powered by language models. It enables applications that:Are context-aware: connect a language model to sources of context (prompt instructions, few shot examples, content to ground its response in, etc.)Reason: rely on a language model to reason (about how to answer based on provided context, what actions to take, etc.)This framework

In [14]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

documents

[Document(page_content='Introduction | 🦜️🔗 Langchain', metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 Langchain', 'description': 'LangChain is a framework for developing applications powered by language models. It enables applications that:', 'language': 'en'}),
 Document(page_content='Skip to main content🦜️🔗 LangChainDocsUse casesIntegrationsGuidesAPIMorePeopleVersioningChangelogContributingTemplatesCookbooksTutorialsYouTube🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS DocsChatSearchGet startedIntroductionInstallationQuickstartSecurityLangChain Expression LanguageGet startedWhy use LCELInterfaceStreamingHow toCookbookLangChain Expression Language (LCEL)ModulesModel I/ORetrievalAgentsChainsMoreLangServeLangSmithLangGraphGet startedIntroductionOn this pageIntroductionLangChain is a framework for developing applications powered by language models. It enables applications that:Are c

In [15]:
vector = FAISS.from_documents(documents, OpenAIEmbeddings())

In [16]:
retriever = vector.as_retriever()

In [17]:
retriever.get_relevant_documents("What is langchain?")[0]

Document(page_content='Introduction | 🦜️🔗 Langchain', metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 Langchain', 'description': 'LangChain is a framework for developing applications powered by language models. It enables applications that:', 'language': 'en'})

In [18]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langchain_search",
    "Search for information about LangChain. For any questions about LangChain, you must use this tool!",
)

In [19]:
tools = [search, retriever_tool]

In [20]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?')

In [21]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [24]:
prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant'))

In [25]:
prompt.messages[1]

MessagesPlaceholder(variable_name='chat_history', optional=True)

In [26]:
prompt.messages[2]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))

In [27]:
prompt.messages[3]

MessagesPlaceholder(variable_name='agent_scratchpad')

In [29]:
tools

[TavilySearchResults(),
 Tool(name='langchain_search', description='Search for information about LangChain. For any questions about LangChain, you must use this tool!', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x128239630>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12e66a620>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x10f8af7f0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12e66a620>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))]

In [28]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [30]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history',

In [40]:


"""
RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')])
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x12f843130>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12f864610>, model_name='gpt-3.5-turbo-0125', temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='https://api.openai.com/v1', openai_proxy=''), kwargs={'functions': [{'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type': 'object', 'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query']}}, {'name': 'langchain_search', 'description': 'Search for information about LangChain. For any questions about LangChain, you must use this tool!', 'parameters': {'type': 'object', 'properties': {'query': {'description': 'query to look up in retriever', 'type': 'string'}}, 'required': ['query']}}]})
| OpenAIFunctionsAgentOutputParser()
"""

"\nRunnableAssign(mapper={\n  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))\n})\n| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_his

In [31]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [32]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [33]:
agent_executor.invoke({"input": "What are some useful applications you can build with LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `langchain_search` with `{'query': 'useful applications of LangChain'}`


Write your applications in LangChain/LangChain.js. Hit the ground running using Templates for reference.Productionize: Use LangSmith to inspect, test and monitor your chains, so that you can constantly improve and deploy with confidence.Deploy: Turn any chain into an API with LangServe.LangChain Libraries​The main value props of the LangChain packages are:Components: composable tools and integrations for working with language models. Components are modular and easy-to-use, whether you are using the rest of the LangChain framework or notOff-the-shelf chains: built-in assemblages of components for accomplishing higher-level tasksOff-the-shelf chains make it easy to get started. Components make it easy to customize existing chains and build new ones.The LangChain libraries themselves are made up of several different packages.langchain-core: Base abstractions and Lan

{'input': 'What are some useful applications you can build with LangChain?',
 'output': 'LangChain can be used to build a variety of useful applications, including:\n\n1. Context-aware applications: LangChain enables applications that are context-aware by connecting a language model to sources of context such as prompt instructions, few-shot examples, and content to ground its response in.\n\n2. Reasoning applications: LangChain allows applications to rely on a language model to reason about how to answer based on provided context and what actions to take.\n\n3. Components and off-the-shelf chains: LangChain provides composable tools and integrations for working with language models, off-the-shelf chains for accomplishing higher-level tasks, and components that make it easy to customize existing chains and build new ones.\n\n4. Deployment as API: LangChain can turn any chain into an API using LangServe, making it easy to deploy applications built with LangChain.\n\n5. LangChain Librari

LANGSMITH!!!